### Imports

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

### Load IOC Data

In [ ]:
with open('scan_iocs.json', 'r') as f:
    iocs = json.load(f)

# Display metadata
print("="*60)
print("ANALYSIS METADATA")
print("="*60)
for key, value in iocs['metadata'].items():
    print(f"{key:20s}: {value}")

### Scan Type Distribution Chart

In [4]:
# Create DataFrame from scan summary
scan_df = pd.DataFrame(list(iocs['scan_summary'].items()), 
                       columns=['Scan Type', 'Count'])

# Filter out UNKNOWN and zero counts
scan_df = scan_df[(scan_df['Count'] > 0) & (scan_df['Scan Type'] != 'UNKNOWN')]

# Create bar chart
plt.figure(figsize=(10, 6))
bars = plt.bar(scan_df['Scan Type'], scan_df['Count'], 
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'])

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.title('Port Scan Type Distribution', fontsize=16, fontweight='bold')
plt.xlabel('Scan Type', fontsize=12)
plt.ylabel('Packet Count', fontsize=12)
plt.tight_layout()
plt.savefig('scan_type_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("[+] Chart saved: scan_type_distribution.png")

NameError: name 'pd' is not defined

### Malicious IP Analysis


In [ ]:
# Create DataFrame from malicious IPs
if iocs['malicious_ips']:
    mal_df = pd.DataFrame(iocs['malicious_ips'])
    
    print("\n" + "="*60)
    print("MALICIOUS IP SUMMARY")
    print("="*60)
    print(mal_df[['ip_address', 'ports_scanned', 'severity', 'avg_packet_rate']])
    
    # Ports scanned chart
    plt.figure(figsize=(10, 6))
    plt.barh(mal_df['ip_address'], mal_df['ports_scanned'], 
             color='#E74C3C', edgecolor='black', linewidth=1.5)
    plt.xlabel('Ports Scanned', fontsize=12)
    plt.ylabel('Source IP', fontsize=12)
    plt.title('Ports Scanned by Source IP', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('ports_scanned_by_ip.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("[+] Chart saved: ports_scanned_by_ip.png")
else:
    print("No malicious IPs detected")

### Timeline Analysis

In [ ]:
from datetime import datetime

if iocs['malicious_ips']:
    for ip_data in iocs['malicious_ips']:
        first = datetime.fromisoformat(ip_data['first_seen'])
        last = datetime.fromisoformat(ip_data['last_seen'])
        duration = ip_data['scan_duration']
        
        print(f"\n{'='*60}")
        print(f"TIMELINE: {ip_data['ip_address']}")
        print(f"{'='*60}")
        print(f"First Packet:    {first.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Last Packet:     {last.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Total Duration:  {duration:.2f} seconds ({duration/60:.2f} minutes)")
        print(f"Packet Rate:     {ip_data['avg_packet_rate']:.2f} packets/sec")
        
        # Classify scan speed
        if ip_data['avg_packet_rate'] < 1.0:
            print(f"Classification:  🐌 SLOW SCAN (Evasion Attempt)")
        elif ip_data['avg_packet_rate'] < 10:
            print(f"Classification:  🚶 MODERATE SCAN")
        else:
            print(f"Classification:  🏃 FAST SCAN")